In [2]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        print('result ', result)
        return result

    return wrapper

@timethis
def countdown(n):
    '''Counts down'''
    while n > 0:
        n -= 1

countdown(100000)
countdown(10000000)

countdown 0.001977205276489258
result  None
countdown 0.209153413772583
result  None


In [8]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end - start)
        return result
    return wrapper

@timethis
def countdown(n:int):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1

countdown(100000)
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)

from inspect import signature
print(signature(countdown))


countdown 0.001909017562866211
countdown

    Counts down
    
{'n': <class 'int'>}
(n: int)


In [11]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging leve, name is the logger name, and message
    is the log message. If name and message aren't specified, they default to the function's
    module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        return wrapper

    return decorate

@logged(logging.DEBUG)
def add(x, y):
    return x + y

add(3, 4)

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

spam()


spam


Spam!


In [5]:
from functools import wraps, partial
import logging

def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper
    return decorate


# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')


import logging
logging.basicConfig(level=logging.DEBUG)
print(add(2, 3))

# change the log message
add.set_message('Add called')
print(add(2, 3))

# change the log level
add.set_level(logging.WARNING)
print(add(2, 3))


DEBUG:__main__:add
DEBUG:__main__:Add called


5
5
5


In [8]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__

    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

@logged
def add(x, y):
    return x + y

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')


In [15]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        if not __debug__:
            return func

        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must {}'.format(name, bound_types[name])
                        )
            return func(*args, **kwargs)
        return wrapper
    return decorate


@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)


spam(1, 2, 3)
spam(1, 'hello', 3)
spam(1, 'hello', 'world')

1 2 3
1 hello 3


TypeError: Argument z must <class 'int'>

In [16]:
from inspect import signature
def spam(x, y, z=42):
    pass

sig = signature(spam)
print(sig)
print(sig.parameters)

(x, y, z=42)
OrderedDict({'x': <Parameter "x">, 'y': <Parameter "y">, 'z': <Parameter "z=42">})


In [17]:
from functools import wraps

class A:
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper

    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            return func(*args, **kwargs)
        return wrapper

a = A()

@a.decorator1
def spam():
    pass

@A.decorator2
def grok():
    pass

spam()
grok()

Decorator 1
Decorator 2


In [20]:
class NoInstance(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("Can't instantiate directly")

class Spam(metaclass=NoInstance):
    @staticmethod
    def grok(x):
        print('Spam.grok')

Spam.grok(42)
s = Spam()

Spam.grok


TypeError: Can't instantiate directly

In [19]:
class Singleton(type):
    def __init__(self, *args, **kwargs):
        self.__instance = None
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs)
            return self.__instance
        else:
            return self.__instance

class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')


a = Spam()
b = Spam()
print(b is a)
c = Spam()
print(a is c)

Creating Spam
True
True


In [22]:
x = 42
print(eval('2 + 3*4 + x'))
exec('for i in range(10): print(i)')

56
0
1
2
3
4
5
6
7
8
9


In [23]:
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
    print('Blastoff!')

import dis
dis.dis(countdown)


  1           0 RESUME                   0

  2           2 LOAD_FAST                0 (n)
              4 LOAD_CONST               1 (0)
              6 COMPARE_OP              68 (>)
             10 POP_JUMP_IF_FALSE       23 (to 58)

  3     >>   12 LOAD_GLOBAL              1 (NULL + print)
             22 LOAD_CONST               2 ('T-minus')
             24 LOAD_FAST                0 (n)
             26 CALL                     2
             34 POP_TOP

  4          36 LOAD_FAST                0 (n)
             38 LOAD_CONST               3 (1)
             40 BINARY_OP               23 (-=)
             44 STORE_FAST               0 (n)

  2          46 LOAD_FAST                0 (n)
             48 LOAD_CONST               1 (0)
             50 COMPARE_OP              68 (>)
             54 POP_JUMP_IF_FALSE        1 (to 58)
             56 JUMP_BACKWARD           23 (to 12)

  5     >>   58 LOAD_GLOBAL              1 (NULL + print)
             68 LOAD_CONST               4 

In [24]:
import sys
print(sys.path)

['C:\\Users\\jiangyz\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\jiangyz\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\jiangyz\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\jiangyz\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\root\\dev\\venv', '', 'c:\\root\\dev\\venv\\Lib\\site-packages', 'c:\\root\\dev\\venv\\Lib\\site-packages\\win32', 'c:\\root\\dev\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\root\\dev\\venv\\Lib\\site-packages\\Pythonwin']


In [26]:
import importlib
math = importlib.import_module('math')
print(math.sin(2))

mod = importlib.import_module('urllib.request')
u = mod.urlopen('http://www.python.org')
print(u)

0.9092974268256817
